In [552]:
#Lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LinearRegression
#from sklearn.feature_selection import
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [518]:
#data load
traindata = pd.read_csv("train.csv",header=1)
testdata = pd.read_csv("test.csv")

In [519]:
traindata.dropna(subset=['Electrical'],inplace=True)

In [520]:
#outlier removal
Q1 = np.percentile(traindata['SalePrice'],25, interpolation ='midpoint')
Q3 = np.percentile(traindata['SalePrice'],75, interpolation ='midpoint')
IQR = Q3-Q1
print(traindata.shape)

(1459, 81)


In [521]:
up=np.where(traindata['SalePrice']>= (Q3+1.5*IQR))
lo=np.where(traindata['SalePrice']<= (Q1-1.5*IQR))

#remove outliers
traindata.drop(up[0], inplace =True)
traindata.drop(lo[0], inplace =True)

#check
print(traindata.shape)

(1398, 81)


In [522]:
train=traindata.values
test=testdata.values

In [523]:
train

array([[1, 60, 'RL', ..., 'WD', 'Normal', 208500],
       [2, 20, 'RL', ..., 'WD', 'Normal', 181500],
       [3, 60, 'RL', ..., 'WD', 'Normal', 223500],
       ...,
       [1458, 70, 'RL', ..., 'WD', 'Normal', 266500],
       [1459, 20, 'RL', ..., 'WD', 'Normal', 142125],
       [1460, 20, 'RL', ..., 'WD', 'Normal', 147500]], dtype=object)

In [524]:
test

array([[1461, 20, 'RH', ..., 2010, 'WD', 'Normal'],
       [1462, 20, 'RL', ..., 2010, 'WD', 'Normal'],
       [1463, 60, 'RL', ..., 2010, 'WD', 'Normal'],
       ...,
       [2917, 20, 'RL', ..., 2006, 'WD', 'Abnorml'],
       [2918, 85, 'RL', ..., 2006, 'WD', 'Normal'],
       [2919, 60, 'RL', ..., 2006, 'WD', 'Normal']], dtype=object)

In [525]:
#imputation
#1.LotFrontage
imputer = SimpleImputer(missing_values=np.nan, strategy = 'mean')
imputer.fit(train[:,3:4])
train[:,3:4]=imputer.transform(train[:,3:4])
#testdata
imputer.fit(test[:,3:4])
test[:,3:4]=imputer.transform(test[:,3:4])

#2.MasVnrArea# constant zero (None)
#imputer = SimpleImputer(missing_values=np.nan, strategy = 'mean')
imputer.fit(train[:,26:27])
train[:,26:27]=imputer.transform(train[:,26:27])
#testdata
imputer.fit(test[:,26:27])
test[:,26:27]=imputer.transform(test[:,26:27])

In [526]:
#replace nan with  NoAlleyAccess
imputer = SimpleImputer(missing_values=np.nan, strategy = 'constant', fill_value='NoAlleyAccess')
imputer.fit(train[:,6:7])
train[:,6:7]=imputer.transform(train[:,6:7])
#testdata
imputer.fit(test[:,6:7])
test[:,6:7]=imputer.transform(test[:,6:7])

In [527]:
#replace nan with  NoBasement
imputer = SimpleImputer(missing_values=np.nan, strategy = 'constant', fill_value='NoBasement')
imputer.fit(train[:,30:34])
train[:,30:34]=imputer.transform(train[:,30:34])
imputer.fit(train[:,35:36])
train[:,35:36]=imputer.transform(train[:,35:36])
#testdata
imputer.fit(train[:,30:34])
test[:,30:34]=imputer.transform(test[:,30:34])
imputer.fit(test[:,35:36])
test[:,35:36]=imputer.transform(test[:,35:36])

In [528]:
#print(train[1:20,30:39])

In [529]:
#replace nan with  NoFireplace
#print(train[1:20,56:58])
imputer = SimpleImputer(missing_values=np.nan, strategy = 'constant', fill_value='NoFireplace')
imputer.fit(train[:,57:58])
train[:,57:58]=imputer.transform(train[:,57:58])
#print(train[1:20,56:58])
#testdata
imputer.fit(test[:,57:58])
test[:,57:58]=imputer.transform(test[:,57:58])

In [530]:
#replace nan with  NoGarage
#print(train[1:50,58:65])

In [531]:
#print(train[1:20,58:59])
imputer = SimpleImputer(missing_values=np.nan, strategy = 'constant', fill_value='NoGarage')
imputer.fit(train[:,58:59])
train[:,58:59]=imputer.transform(train[:,58:59])
imputer.fit(train[:,60:61])
train[:,60:61]=imputer.transform(train[:,60:61])
imputer.fit(train[:,63:65])
train[:,63:65]=imputer.transform(train[:,63:65])
#testdata
imputer.fit(test[:,58:59])
test[:,58:59]=imputer.transform(test[:,58:59])
imputer.fit(test[:,60:61])
test[:,60:61]=imputer.transform(test[:,60:61])
imputer.fit(test[:,63:65])
test[:,63:65]=imputer.transform(test[:,63:65])

imputer = SimpleImputer(missing_values=np.nan, strategy = 'constant', fill_value='0000')
imputer.fit(train[:,59:60])
train[:,59:60]=imputer.transform(train[:,59:60])
#print(train[1:20,58:65])
#testdata
imputer.fit(test[:,59:60])
test[:,59:60]=imputer.transform(test[:,59:60])


In [556]:
#print(train[1:50,58:65])
traindata.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1398.000000,1398.000000,1142.000000,1398.000000,1398.000000,1398.000000,1398.000000,1398.000000,1391.000000,1398.000000,...,1398.000000,1398.000000,1398.000000,1398.000000,1398.000000,1398.000000,1398.000000,1398.000000,1398.000000,1398.000000
mean,733.047926,57.449928,69.070928,10162.603004,5.989270,5.589413,1970.145923,1984.109442,90.662832,418.970672,...,89.694564,44.988555,22.394134,3.451359,14.644492,2.484263,45.417740,6.319027,2007.821888,170607.909871
std,422.611408,42.825945,23.847221,8328.819437,1.285022,1.118057,30.002389,20.655373,159.759119,429.705379,...,121.878105,65.718200,61.473553,29.683732,54.770297,38.303325,506.926231,2.708416,1.328249,59789.261939
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,367.250000,20.000000,58.250000,7440.000000,5.000000,5.000000,1953.000000,1966.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129000.000000
50%,738.500000,50.000000,68.000000,9328.500000,6.000000,5.000000,1971.000000,1992.000000,0.000000,370.500000,...,0.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,159697.500000
75%,1097.750000,70.000000,80.000000,11338.250000,7.000000,6.000000,1999.000000,2003.000000,145.000000,686.000000,...,165.750000,64.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,204562.500000
max,1460.000000,190.000000,313.000000,164660.000000,10.000000,9.000000,2009.000000,2010.000000,1600.000000,5644.000000,...,736.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,394617.000000


In [533]:
#replace nan with  NoPool
#print(train[1:20,72:73])
imputer = SimpleImputer(missing_values=np.nan, strategy = 'constant', fill_value='NoPool')
imputer.fit(train[:,72:73])
train[:,72:73]=imputer.transform(train[:,72:73])
#print(train[1:20,72:73])
#testdata
imputer.fit(test[:,72:73])
test[:,72:73]=imputer.transform(test[:,72:73])

In [534]:
#replace nan with  NoFence
#print(train[1:20,73:74])
imputer = SimpleImputer(missing_values=np.nan, strategy = 'constant', fill_value='NoFence')
imputer.fit(train[:,73:74])
train[:,73:74]=imputer.transform(train[:,73:74])
#print(train[1:20,73:74])
#testdata
imputer.fit(test[:,73:74])
test[:,73:74]=imputer.transform(test[:,73:74])

In [535]:
#replace nan with  NoMiscFeature
#print(train[1:20,74:75])
imputer = SimpleImputer(missing_values=np.nan, strategy = 'constant', fill_value='NoMiscFeature')
imputer.fit(train[:,74:75])
train[:,74:75]=imputer.transform(train[:,74:75])
#print(train[1:20,74:75])
#testdata
imputer.fit(test[:,74:75])
test[:,74:75]=imputer.transform(test[:,74:75])

In [536]:
train_cata=pd.DataFrame(train,columns=traindata.columns)
test_cata=pd.DataFrame(test,columns=testdata.columns)

In [537]:
y_train=train_cata.iloc[:,-1].values
y_test=test_cata.iloc[:,:].values
#y
c = train_cata.iloc[:, 1:80]
#c
c_test= test_cata.iloc[:, 1:80]
c_test


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,120,0,NoPool,MnPrv,NoMiscFeature,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NoAlleyAccess,IR1,Lvl,AllPub,Corner,...,0,0,NoPool,NoFence,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NoAlleyAccess,IR1,Lvl,AllPub,Inside,...,0,0,NoPool,MnPrv,NoMiscFeature,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NoAlleyAccess,IR1,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMiscFeature,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NoAlleyAccess,IR1,HLS,AllPub,Inside,...,144,0,NoPool,NoFence,NoMiscFeature,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMiscFeature,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMiscFeature,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMiscFeature,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,MnPrv,Shed,700,7,2006,WD,Normal


In [538]:
train_cata1=c
test_cata1=c_test

In [539]:
train_cata1

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMiscFeature,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NoAlleyAccess,Reg,Lvl,AllPub,FR2,...,0,0,NoPool,NoFence,NoMiscFeature,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NoAlleyAccess,IR1,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMiscFeature,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NoAlleyAccess,IR1,Lvl,AllPub,Corner,...,0,0,NoPool,NoFence,NoMiscFeature,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NoAlleyAccess,IR1,Lvl,AllPub,FR2,...,0,0,NoPool,NoFence,NoMiscFeature,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393,60,RL,62.0,7917,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMiscFeature,0,8,2007,WD,Normal
1394,20,RL,85.0,13175,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,MnPrv,NoMiscFeature,0,2,2010,WD,Normal
1395,70,RL,66.0,9042,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,GdPrv,Shed,2500,5,2010,WD,Normal
1396,20,RL,68.0,9717,Pave,NoAlleyAccess,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMiscFeature,0,4,2010,WD,Normal


In [540]:
#Encode categorical data
catag_columns=['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig','LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType','ExterQual', 'ExterCond', 'Foundation', 'BsmtQual','BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
       'BsmtFinType2', 'Heating','HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual','GarageCond', 'PavedDrive', 'PoolQC','Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
for col in catag_columns:
    col_cat = pd.get_dummies(c[col], prefix=col)
    train_cata1 =pd.concat((train_cata1, col_cat),axis=1).drop(col,axis=1)
    test_cata1 =pd.concat((test_cata1, col_cat),axis=1).drop(col,axis=1)
X_train=train_cata1.iloc[:,:].values
X_test=test_cata1.iloc[:,:].values

In [541]:
#pd.set_option('display.max_seq_items',None)
#train_cata1.columns
#X_train.shape()
#train_cata1
X_train
train_cata1.to_excel("X_train.xlsx")  

In [542]:
#score, pvalues

In [543]:
#apply regression
regressor=LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [554]:
#predict and test
#X_test

In [546]:
#y_pred_reg = regressor.predict(X_test)



In [550]:
#DecisionTree
DTregressor = DecisionTreeRegressor(random_state = 0)
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [553]:
#RandomForest
RFRegressor=RandomForestRegressor(n_estimators=10, random_state=0)
RFRegressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=0)